## Youtube adview prediction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
dataset=pd.read_csv('train.csv')
dataset

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.dtypes

In [ ]:
#Visulaization
#Individual plots
plt.hist(dataset["category"])

In [ ]:
plt.plot(dataset['adview'])
plt.show()

In [ ]:
#Heatmap
import seaborn as sns
f,ax=plt.subplots(figsize=(10,8))
corr=dataset.corr()
sns.heatmap(corr,mask=np.zeros_like(corr,dtype=np.bool),cmap=sns.diverging_palette(220,10,as_cmap=True),
            square=True,ax=ax,annot=True)

plt.show()

In [ ]:
#Removing character "F" in data
dataset=dataset[dataset.views!='F']
dataset=dataset[dataset.likes!='F']
dataset=dataset[dataset.dislikes!='F']
dataset=dataset[dataset.comment!='F']
dataset



In [ ]:
dataset.head()

In [ ]:
#Assigning each category a number fo category features
category={"A":1,"B":2,"C":3,"D":4,"E":5,"F":6,"G":7,"H":8}
dataset["category"]=dataset["category"].map(category)

In [ ]:
dataset.shape

In [ ]:
#Convert Values to integers for views,likes,comments,dislikes and adview
dataset["views"]=pd.to_numeric(dataset["views"])
dataset["comment"]=pd.to_numeric(dataset["comment"])
dataset["likes"]=pd.to_numeric(dataset["likes"])
dataset["dislikes"]=pd.to_numeric(dataset["dislikes"])
dataset["adview"]=pd.to_numeric(dataset["adview"])

In [ ]:
column_vid=dataset["vidid"]

In [ ]:
#Encoing features like categoty,duration,vidid
from sklearn.preprocessing import LabelEncoder
dataset['duration']=LabelEncoder().fit_transform(dataset['duration'])
dataset['vidid']=LabelEncoder().fit_transform(dataset['vidid'])
dataset['published']=LabelEncoder().fit_transform(dataset['published'])

In [ ]:
dataset.head()

In [ ]:
import datetime
import time

In [ ]:
#cONVERT TIME IN SECONDS FOR DURATION
def checki(x):
    y = x[2:]
    h = ''
    m = ''
    s = ''
    mm = ''
    P = ['H','M','S']
    for i in y:
        if i not in P:
            mm+=i
        else:
            if(i=="H"):
                h = mm
                mm = ''
            elif(i == "M"):
                m = mm
                mm = ''
            else:
                s = mm
                mm = ''
        if(h==''):
            h = '00'
        if(m == ''):
            m = '00'
        if(s==''):
            s='00'
    bp = h+':'+m+':'+s
    return bp
train=pd.read_csv("train.csv")
mp = pd.read_csv("train.csv")["duration"]
time = mp.apply(checki)
def func_sec(time_string):
    h, m, s = time_string.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)
time1=time.apply(func_sec)
dataset["duration"]=time1
dataset.head()

In [ ]:
#Visulaization
#Individual plots
plt.hist(dataset["category"])

In [ ]:
plt.show()


In [ ]:
plt.plot(dataset['adview'])
plt.show()



In [ ]:
#Remove videos which adview is greater than 2000000 as outlier
dataset=dataset[dataset["adview"]<2000000]

In [ ]:
dataset.shape

In [ ]:
#Heatmap
import seaborn as sns
f,ax=plt.subplots(figsize=(10,8))
corr=dataset.corr()
sns.heatmap(corr,mask=np.zeros_like(corr,dtype=np.bool),cmap=sns.diverging_palette(220,10,as_cmap=True),
            square=True,ax=ax,annot=True)

plt.show()

In [ ]:
dataset.head()

In [ ]:
#Split data
Y_train = pd.DataFrame(data = dataset.iloc[:, 1].values, columns= ['target'])
dataset=dataset.drop(['adview'],axis=1)
dataset=dataset.drop(["vidid"],axis=1)
dataset.head()


In [ ]:
dataset.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(dataset,Y_train,test_size=0.2,random_state=42)
X_train.shape

In [ ]:
#Normalize data
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)
X_train.mean()

In [ ]:
#Evaluation Metrics
from sklearn import metrics
def print_error(X_test,y_test,model_name):
    prediction=model_name.predict(X_test)
    print('Mean Absolute error:',metrics.mean_absolute_error(y_test,prediction))
    print('Mean Squared error:',metrics.mean_squared_error(y_test,prediction))
    print('Root mean squared error:',np.sqrt(metrics.mean_squared_error(y_test,prediction)))

In [ ]:
#LinearRegression
from sklearn import linear_model
linear_regression=linear_model.LinearRegression()
linear_regression.fit(X_train,y_train)
print_error(X_test,y_test,linear_regression)

In [ ]:
#Decision tree regressor
from sklearn.tree import DecisionTreeRegressor
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
print_error(X_test,y_test, decision_tree)



In [ ]:
#Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
n_estimators=200
max_depth=25
min_samples_split=15
min_samples_leaf=2
random_forest = RandomForestRegressor(n_estimators = n_estimators,max_depth = max_depth, min_samples_split=15,min_samples_leaf=2)
random_forest.fit(X_train,np.ravel(y_train))
print_error(X_test,y_test, random_forest)

In [ ]:
#Support Vector Regressor
from sklearn.svm import SVR
supportvector_regressor=SVR()
supportvector_regressor.fit(X_train,np.ravel(y_train))
print_error(X_test,y_test,supportvector_regressor)

In [ ]:

# Artificial Neural Network
import keras
from keras.layers import Dense


ann = keras.models.Sequential([
    Dense(6, activation="relu",
    input_shape=X_train.shape[1:]),
    Dense(6,activation="relu"),
    Dense(1)
])
ptimizer=keras.optimizers.Adam()
loss=keras.losses.mean_squared_error
ann.compile(optimizer=optimizer,loss=loss,metrics=["mean_squared_error"])
history=ann.fit(X_train,y_train,epochs=100)
ann.summary()
print_error(X_test,y_test,ann)

In [ ]:
ann.summary()


In [ ]:
print_error(X_test,y_test,ann)

In [ ]:
#Saving scikitlearn models
import joblib
joblib.dump(decision_tree,"decisiontree_youtubeadview.pkl")

In [ ]:
#Saving keras artificial neural networks model
ann.save("ann_youtubeadview.h5")

# Test.csv

In [ ]:

df_test = pd.read_csv('test.csv')

In [ ]:
df_test

In [ ]:
from keras.models import load_model
m = load_model("ann_youtubeadview.h5")

In [ ]:
df_test.shape

In [ ]:
#Removing character "F" in data
df_test=df_test[df_test.views!='F']
df_test=df_test[df_test.likes!='F']
df_test=df_test[df_test.dislikes!='F']
df_test=df_test[df_test.comment!='F']
df_test

In [ ]:
#Assigning each category a number fo category features
category={"A":1,"B":2,"C":3,"D":4,"E":5,"F":6,"G":7,"H":8}
df_test["category"]=df_test["category"].map(category)
df_test

In [ ]:
#Convert Values to integers for views,likes,comments,dislikes and adview
df_test["views"]=pd.to_numeric(df_test["views"])
df_test["comment"]=pd.to_numeric(df_test["comment"])
df_test["likes"]=pd.to_numeric(df_test["likes"])
df_test["dislikes"]=pd.to_numeric(df_test["dislikes"])

In [ ]:
column_vid=df_test["vidid"]

In [ ]:
#Encoing features like categoty,duration,vidid
from sklearn.preprocessing import LabelEncoder
df_test['duration']=LabelEncoder().fit_transform(df_test['duration'])
df_test['vidid']=LabelEncoder().fit_transform(df_test['vidid'])
df_test['published']=LabelEncoder().fit_transform(df_test['published'])

In [ ]:
df_test.head()

In [ ]:
import datetime
import time

In [ ]:
#cONVERT TIME IN SECONDS FOR DURATION
def checki(x):
    y = x[2:]
    h = ''
    m = ''
    s = ''
    mm = ''
    P = ['H','M','S']
    for i in y:
        if i not in P:
            mm+=i
        else:
            if(i=="H"):
                h = mm
                mm = ''
            elif(i == "M"):
                m = mm
                mm = ''
            else:
                s = mm
                mm = ''
        if(h==''):
            h = '00'
        if(m == ''):
            m = '00'
        if(s==''):
            s='00'
    bp = h+':'+m+':'+s
    return bp
train=pd.read_csv("test.csv")
mp = pd.read_csv("test.csv")["duration"]
time = mp.apply(checki)
def func_sec(time_string):
    h, m, s = time_string.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)
time1=time.apply(func_sec)
df_test["duration"]=time1
df_test.head()

In [ ]:
#Visulaization
#Individual plots
plt.hist(df_test["category"])

In [ ]:
plt.plot(df_test['published'])
plt.show()

In [ ]:
#Heatmap
import seaborn as sns
f,ax=plt.subplots(figsize=(10,8))
corr=df_test.corr()
sns.heatmap(corr,mask=np.zeros_like(corr,dtype=np.bool),cmap=sns.diverging_palette(220,10,as_cmap=True),
            square=True,ax=ax,annot=True)

plt.show()

In [ ]:
df_test=df_test.drop(["vidid"],axis=1)
df_test.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_test = df_test
X_test=scaler.fit_transform(X_test)

In [ ]:
prediction = m.predict(X_test)

In [ ]:
prediction=pd.DataFrame(prediction)
prediction.info()

In [ ]:
prediction = prediction.rename(columns={0: "Adview"})

In [ ]:
prediction

In [ ]:
prediction.to_csv('predictions.csv')